# 1. Set up the environment

Before we can import the required libraries, we need to install them.
You can install the libraries with pip and the following commands:
 pip install refinitiv-dataplatform
 pip install pandas

In [1]:
import refinitiv.dataplatform.eikon as ek
import refinitiv.data as rd
import pandas as pd
import time

#https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#esg-data-in-python

Setting up the configuration for Refinitiv API is tidious. Here are two different ways:

Define the session using your own credentials store.
It is best practice to store your credentials in a separate file and load them from there.

You will get your password and username from the university library.
The App key you will have to generate in the APP Key Generator in the Refinitiv Workspace (Desktop Application or Website).

In [2]:
import passwords
RDP_LOGIN = passwords.RDP_LOGIN
RDP_PASSWORD = passwords.RDP_PASSWORD
APP_KEY = passwords.APP_KEY

In [3]:
session = rd.session.platform.Definition(
    app_key = APP_KEY,
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
).get_session()

/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/data/session/platform.py:90:FutureWarning: 
You open a platform session using the default value of the signon_control parameter (signon_control=True).
In future library version v2.0, this default will be changed to False.
If you want to keep the same behavior as today, you will need to set the signon_control parameter to True either in the library configuration file
({'sessions':{'platform':{'your_session_name':{'signon_control':true}}}}) or in your code where you create the Platform Session.
These alternative options are already supported in the current version of the library.


In [4]:
session.open()

<OpenState.Opened: 'Opened'>

You can also setup a configuration file and specify the sessions in there. I specified the platform session in the configuration file as default
https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/blob/main/Configuration/refinitiv-data.config.json

In [3]:
rd.open_session()

/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/data/_access_layer/session.py:71:FutureWarning: 
You open a platform session using the default value of the signon_control parameter (signon_control=True).
In future library version v2.0, this default will be changed to False.
If you want to keep the same behavior as today, you will need to set the signon_control parameter to True either in the library configuration file
({'sessions':{'platform':{'your_session_name':{'signon_control':true}}}}) or in your code where you create the Platform Session.
These alternative options are already supported in the current version of the library.


<refinitiv.data.session.Definition object at 0x1608cc4d0 {name='rdp'}>

Some functionality is only provided in Eikon, so we set up Eikon access as well.
The Eikon Data API needs the Eikon/Workspace Desktop application as the API proxy interface between the API and the Eikon Platform. For this reason, the Eikon/Workspace application must be running when you use the Eikon Data API Python library.

In [5]:
ek.set_app_key(APP_KEY)

# 2. Get accounting data

For our first example we are interested in the price of the S&P 500 constituents.
We start by downloading the unique identifiers (RIC in the Refinitv world) of the constituents.

Refinitiv Data Platform is the new strategic data distribution platform aimed to unify the distribution of all Refinitiv content assets. However, at this time numerous datasets available through Eikon are not yet available through RDPs.
Using RDP Libraries you can access RDP and retrieve data that is already available on this platform, and you can also utilize Eikon Data APIs and retrieve data that is not yet available from RDP.

Lets start by looking at some fundamental data
We can use different identifiers for companies (RIC, ISIN and CUSIP)

In [10]:
rd.get_data(
    universe=['AAPL.OQ'],
    fields=['TR.Revenue']
)

,Instrument,Revenue
0,AAPL.OQ,383285000000


To get the "code" for our variables of interest, we use the Data Item Browser in the Refinitiv Workspace.

In [6]:
rd.get_data(
    universe=['AAPL.OQ'],
    fields=['TR.Revenue', 'TR.F.TotAssets', 'TR.F.TotLiab', 'TR.TURNOVER']
)

,Instrument,Revenue,Total Assets,Total Liabilities,Turnover
0,AAPL.OQ,383285000000,352583000000,290437000000,3113009126


We can also look at different time periods

In [7]:
rd.get_data(
    universe=['AAPL.OQ'],
    fields=['TR.Revenue.date', 'TR.Revenue', 'TR.F.TotAssets', 'TR.F.TotLiab'],
    parameters = {'SDate':0, 'EDate':-3}
)

,Instrument,Date,Revenue,Total Assets,Total Liabilities
0,AAPL.OQ,2023-09-30,383285000000,352583000000,290437000000
1,AAPL.OQ,2022-09-24,394328000000,352755000000,302083000000
2,AAPL.OQ,2021-09-25,365817000000,351002000000,287912000000
3,AAPL.OQ,2020-09-26,274515000000,323888000000,258549000000


# 3. Get price data

Let's have a look at some pricing information.
Get Apples market capitalization for the last 10 days

In [8]:
rd.get_data(
    universe=['AAPL.OQ'],
    fields=['TR.CompanyMarketCapitalization.date', 'TR.CompanyMarketCapitalization'],
    parameters = {'SDate':0, 'EDate':-10, 'Frq':'D'}
)

,Instrument,Date,Company Market Capitalization
0,AAPL.OQ,2024-02-22,2833276325880
1,AAPL.OQ,2024-02-21,2815363743920
2,AAPL.OQ,2024-02-20,2803627914360
3,AAPL.OQ,2024-02-16,2815209325110
4,AAPL.OQ,2024-02-15,2839144240660
5,AAPL.OQ,2024-02-14,2843622386150
6,AAPL.OQ,2024-02-13,2857365660240
7,AAPL.OQ,2024-02-12,2889948029150
8,AAPL.OQ,2024-02-09,2916199226850
9,AAPL.OQ,2024-02-08,2908015029920


We can use calendar dates as well

In [9]:
rd.get_data(
    universe=['AAPL.OQ'],
    fields=['TR.CompanyMarketCapitalization.date', 'TR.CompanyMarketCapitalization'],
    parameters = {'SDate':'2024-01-01', 'EDate':'2024-02-21', 'Frq':'D'}
)

,Instrument,Date,Company Market Capitalization
0,AAPL.OQ,2024-01-02,2870346373440
1,AAPL.OQ,2024-01-03,2848854338000
2,AAPL.OQ,2024-01-04,2812673501360
3,AAPL.OQ,2024-01-05,2801386317280
4,AAPL.OQ,2024-01-08,2869109421760
5,AAPL.OQ,2024-01-09,2862615425440
6,AAPL.OQ,2024-01-10,2878850416240
7,AAPL.OQ,2024-01-11,2869573278640
8,AAPL.OQ,2024-01-12,2874675704320
9,AAPL.OQ,2024-01-16,2839267962480


We can also include other variables of interest

In [18]:
rd.get_data(
    universe=['AAPL.OQ'],
    fields=['TR.CompanyMarketCapitalization.date', 'TR.CompanyMarketCapitalization', 'TR.BIDPRICE', 'TR.ASKPRICE', 'TR.HIGHPRICE', 'TR.LOWPRICE'],
    parameters = {'SDate':'2024-01-01', 'EDate':'2024-02-21', 'Frq':'D'}
)

,Instrument,Date,Company Market Capitalization,Bid Price,Ask Price,High Price,Low Price
0,AAPL.OQ,2024-01-02,2870346373440,185.5,185.53,188.43,183.89
1,AAPL.OQ,2024-01-03,2848854338000,184.23,184.25,185.87,183.44
2,AAPL.OQ,2024-01-04,2812673501360,181.9,181.92,183.08,180.88
3,AAPL.OQ,2024-01-05,2801386317280,181.17,181.19,182.76,180.18
4,AAPL.OQ,2024-01-08,2869109421760,185.54,185.56,185.59,181.5
5,AAPL.OQ,2024-01-09,2862615425440,185.12,185.13,185.15,182.73
6,AAPL.OQ,2024-01-10,2878850416240,186.14,186.18,186.395,183.92
7,AAPL.OQ,2024-01-11,2869573278640,185.58,185.59,187.04,183.62
8,AAPL.OQ,2024-01-12,2874675704320,185.91,185.92,186.73,185.19
9,AAPL.OQ,2024-01-16,2839267962480,183.6,183.61,184.255,180.94


Refinity offers the "get_history()" function to retrieve historical price data.

In [27]:
rd.get_history(universe='AAPL.OQ')

AAPL.OQ,TRDPRC_1,HIGH_1,LOW_1,ACVOL_UNS,OPEN_PRC,BID,ASK,TRNOVR_UNS,VWAP,BLKCOUNT,BLKVOLUM,NUM_MOVES,TRD_STATUS,SALTIM,CLS_AUCVOL,OPN_AUCVOL
Date,,,,,,,,,,,,,,,,
2024-01-23,195.18,195.75,193.83,11003516,195.0,195.14,195.16,2144676378,194.926,3,3945051,93574,1,75600,3296641,584185
2024-01-24,194.5,196.37,194.35,14498516,195.505,194.5,194.52,2827058931,194.991,7,6496101,104094,1,75600,5873072,515743
2024-01-25,194.17,196.26,193.115,15502005,195.2,194.17,194.19,3017260026,194.6412,3,5813491,123890,1,75600,5127348,675524
2024-01-26,192.42,194.76,191.94,13893779,194.17,192.4,192.42,2679862560,192.8731,2,6283302,93157,1,75600,5737336,545966
2024-01-29,191.73,192.195,189.59,14005433,191.99,191.78,191.8,2681933991,191.4875,6,7200007,95459,1,75600,6563975,568661
2024-01-30,188.04,191.8,187.47,13379033,190.84,187.97,187.99,2524483569,188.6887,3,5454821,123771,1,75600,4912224,532597
2024-01-31,184.4,187.0,184.36,17757463,187.0,184.38,184.45,3286014090,185.0326,2,9262813,132781,1,75600,8626440,636373
2024-02-01,186.86,186.95,183.82,16281081,183.9,186.89,186.9,3022736501,186.2064,10,5862336,145874,1,75600,5073836,651567
2024-02-02,185.85,187.31,179.25,27604918,179.96,185.83,185.87,5096284452,184.7255,18,10457155,234042,1,75600,8319880,1894683


For analyses of multiple companies we can provide a list of companies

In [22]:
rd.get_data(
    universe=['AAPL.OQ', 'TSLA.O'],
    fields=['TR.CompanyMarketCapitalization.date', 'TR.CompanyMarketCapitalization'],
    parameters = {'SDate':0, 'EDate':-10, 'FRQ':'D'}
)

A:\Data\GermanBusinessPanelTeam\Schrader\PycharmProjects\TutorialACC540\venv\lib\site-packages\pandas\core\dtypes\cast.py:1057:RuntimeWarning: invalid value encountered in cast
A:\Data\GermanBusinessPanelTeam\Schrader\PycharmProjects\TutorialACC540\venv\lib\site-packages\pandas\core\dtypes\cast.py:1081:RuntimeWarning: invalid value encountered in cast


,Instrument,Date,Company Market Capitalization
0,AAPL.OQ,2024-02-20,2803627914360.0
1,AAPL.OQ,2024-02-16,2815209325110.0
2,AAPL.OQ,2024-02-15,2839144240660.0
3,AAPL.OQ,2024-02-14,2843622386150.0
4,AAPL.OQ,2024-02-13,2857365660240.0
5,AAPL.OQ,2024-02-12,2889948029150.0
6,AAPL.OQ,2024-02-09,2916199226850.0
7,AAPL.OQ,2024-02-08,2908015029920.0
8,AAPL.OQ,2024-02-07,2924846680210.0
9,AAPL.OQ,2024-02-06,2923148073300.0


We could also look at all companies in a given index.
We start by getting all identifiers from companies listed in the S&P 500.

In [8]:
sp500 = ek.get_data(
    instruments=".SPX",
    fields=["TR.IndexConstituentRIC","TR.IndexConstituentName"],
    parameters={'SDate':'2024-01-01'})[0]['Constituent RIC'].tolist()

/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/dataplatform/legacy/data_grid.py:284:FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead


In [9]:
sp500[:10]

['POOL.OQ',
 'CHRW.OQ',
 'AJG.N',
 'CNP.N',
 'AMCR.N',
 'WM.N',
 'BA.N',
 'FOX.OQ',
 'WY.N',
 'MCD.N']

I theory we can submit a list of all instruments of interest, but often the API call will fail if the list is too long.

In [21]:
price = rd.get_history(
    universe=sp500,
    start = '2024-01-01',
    end = '2024-02-21',
    interval = 'daily',
    fields=['OPEN_PRC', 'TRDPRC_1'],
    adjustments='exchangeCorrection')

RDError: Error code -1 | No data to return, please check errors: ERROR: No successful response.
(401, token expired)

In [10]:
price = rd.get_history(
    universe=['AAPL.OQ', 'TSLA.O'],
    start = '2024-01-01',
    end = '2024-02-21',
    interval = 'daily',
    fields=['OPEN_PRC', 'TRDPRC_1'],
    adjustments='exchangeCorrection')

In [11]:
price

AAPL.OQ            TSLA.O         
           OPEN_PRC TRDPRC_1 OPEN_PRC TRDPRC_1
Date                                          
2024-01-02   187.03   185.64   250.08   248.42
2024-01-03    184.2   184.25   244.98   238.45
2024-01-04    182.0   181.91   239.25   237.93
2024-01-05    181.9   181.18   236.86   237.49
2024-01-08    182.0   185.56   236.14   240.45
2024-01-09   183.96   185.14   238.11   234.96
2024-01-10   184.28   186.19    235.1   233.94
2024-01-11    186.6   185.59   230.57   227.22
2024-01-12    186.0   185.92   220.08   218.89
2024-01-16   182.23   183.63    215.1   219.91
2024-01-17   181.18   182.68   214.86   215.55
2024-01-18   186.01   188.63   216.88   211.88
2024-01-19   189.47   191.56   209.99   212.19
2024-01-22   192.34   193.89   212.26    208.8
2024-01-23    195.0   195.18    211.3   209.14
2024-01-24  195.505    194.5   211.88   207.83
2024-01-25    195.2   194.17    189.7   182.63
2024-01-26   194.17   192.42    185.5   183.25
2024-01-29   191.99   191.73   185.63   190.93
2024-01-30   190.84   188.04   195.33   191.59
2024-01-31    187.0    184.4  186.995   187.29
2024-02-01    183.9   186.86    188.5   188.86
2024-02-02   179.96   185.85   185.04   187.91
2024-02-05   188.07   187.68   184.26   181.06
2024-02-06   186.85    189.3   177.21    185.1
2024-02-07   190.51   189.41   188.18   187.58
2024-02-08   189.34   188.32    189.0   189.56
2024-02-09   188.75   188.85   190.18   193.57
2024-02-12   188.35   187.15   192.11   188.13
2024-02-13   185.81   185.04   183.99   184.02
2024-02-14    185.3   184.15    185.3   188.71
2024-02-15   183.55   183.86   189.16   200.45
2024-02-16   183.62   182.31   202.06   199.95
2024-02-20   181.74   181.56   196.13   193.76
2024-02-21   182.05   182.32   193.36   194.77

Split up the data in chunks for better performance.

In [12]:
# Get ESG data for all CUSIPs
chunk_size = 50
sp500_chunks = [sp500[i:i + chunk_size] for i in range(0, len(sp500), chunk_size)]

# Initialize an empty DataFrame to store the combined results
all_data_list = []

# Iterate over each chunk and perform the API call
for i, chunk in enumerate(sp500_chunks):
    while True:
        try:
            # Dummy API call function (Replace with actual API call)
            chunk_data = rd.get_history(
                universe=chunk,
                start = '2024-01-01',
                end = '2024-02-21',
                interval = 'daily',
                fields=['OPEN_PRC', 'TRDPRC_1'],
                adjustments='exchangeCorrection')

            # Combine the data
            all_data_list.append(chunk_data)
            break  # Exit the loop if API call is successful

        except Exception as e:
            print(e)
            time.sleep(60)

    # Print the progress
    print(f"Processed chunk {i + 1}/{len(sp500_chunks)}")

Processed chunk 1/11
Processed chunk 2/11


/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processed chunk 3/11
Processed chunk 4/11
Processed chunk 5/11
Processed chunk 6/11
Processed chunk 7/11
Processed chunk 8/11
Processed chunk 9/11
Processed chunk 10/11
Processed chunk 11/11


In [13]:
df = pd.concat(all_data_list)
df

POOL.OQ           CHRW.OQ             AJG.N             CNP.N  \
           OPEN_PRC TRDPRC_1 OPEN_PRC TRDPRC_1 OPEN_PRC TRDPRC_1 OPEN_PRC   
Date                                                                        
2024-01-02   394.26   388.15    86.43    87.13   224.86    225.0    28.52   
2024-01-03   382.87   373.78     87.0    87.34    225.6   225.02    29.02   
2024-01-04   370.33   372.87     87.0    88.21   226.55    225.9    29.25   
2024-01-05    371.6   377.73    87.85    88.59   226.42   225.28    29.03   
2024-01-08   374.63   381.44    88.37    87.59   226.65   228.84    29.19   
...             ...      ...      ...      ...      ...      ...      ...   
2024-02-14     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
2024-02-15     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
2024-02-16     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
2024-02-20     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   
2024-02-21     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   

                      AMCR.N           ...    PSX.N            SCHW.N  \
           TRDPRC_1 OPEN_PRC TRDPRC_1  ... OPEN_PRC TRDPRC_1 OPEN_PRC   
Date                                   ...                              
2024-01-02    29.01     9.64     9.75  ...     <NA>     <NA>     <NA>   
2024-01-03    29.13     9.65      9.6  ...     <NA>     <NA>     <NA>   
2024-01-04    29.08     9.56     9.64  ...     <NA>     <NA>     <NA>   
2024-01-05    29.24     9.58     9.74  ...     <NA>     <NA>     <NA>   
2024-01-08    29.41     9.67     9.81  ...     <NA>     <NA>     <NA>   
...             ...      ...      ...  ...      ...      ...      ...   
2024-02-14     <NA>     <NA>     <NA>  ...   146.21   145.21    63.16   
2024-02-15     <NA>     <NA>     <NA>  ...    144.5   146.37     63.4   
2024-02-16     <NA>     <NA>     <NA>  ...   145.33   143.81    64.44   
2024-02-20     <NA>     <NA>     <NA>  ...   143.65   141.14    63.93   
2024-02-21     <NA>     <NA>     <NA>  ...   141.46   143.55    63.25   

                     SNPS.OQ               J.N             AVY.N           
           TRDPRC_1 OPEN_PRC TRDPRC_1 OPEN_PRC TRDPRC_1 OPEN_PRC TRDPRC_1  
Date                                                                       
2024-01-02     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>  
2024-01-03     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>  
2024-01-04     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>  
2024-01-05     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>  
2024-01-08     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>  
...             ...      ...      ...      ...      ...      ...      ...  
2024-02-14    63.03   554.59   562.73   144.13   144.29   204.75   206.39  
2024-02-15    64.63   567.14   557.83   144.79    147.3   206.94   208.67  
2024-02-16     64.4   559.95   552.91    147.3   147.54   208.33   208.83  
2024-02-20    63.58    551.5   547.98   146.26   145.37   207.76   209.71  
2024-02-21    64.13    539.0   543.57   145.23   144.56   210.01   210.93  

[385 rows x 1010 columns]

... or iterate over all companies

In [14]:
idx = pd.date_range('2024-01-01' , '2024-02-21') # We need a continuous calendar data dataset in order to perform merges later
prices = pd.DataFrame()

# Iterate over each chunk and perform the API call
for i, ric in enumerate(sp500):
    print(ric)
    while True:
        try:
            # Dummy API call function (Replace with actual API call)
            price = rd.get_history(
                universe=ric,
                start = '2024-01-01',
                end = '2024-02-21',
                interval = 'daily',
                fields=['OPEN_PRC', 'TRDPRC_1'],
                adjustments='exchangeCorrection')
            price = price.reindex(idx)
            price['RIC'] = ric
            prices = pd.concat([prices, price])

            break  # Exit the loop if API call is successful

        except Exception as e:
            print(e)
            time.sleep(60)

    # Print the progress
    print(f"Processed company {i + 1}/{len(sp500)}")

POOL.OQ
Processed company 1/505
CHRW.OQ
Processed company 2/505
AJG.N
Processed company 3/505
CNP.N
Processed company 4/505
AMCR.N
Processed company 5/505
WM.N
Processed company 6/505
BA.N
Processed company 7/505
FOX.OQ
Processed company 8/505
WY.N
Processed company 9/505
MCD.N
Processed company 10/505
HD.N
Processed company 11/505
SPG.N
Processed company 12/505
HBAN.OQ
Processed company 13/505
QRVO.OQ
Processed company 14/505
LVS.N
Processed company 15/505
CTRA.N
Processed company 16/505
ON.OQ
Processed company 17/505
AIG.N
Processed company 18/505
AIZ.N
Processed company 19/505
CARR.N
Processed company 20/505
EVRG.OQ
Processed company 21/505
COST.OQ
Processed company 22/505
MCO.N
Processed company 23/505
DIS.N
Processed company 24/505
LULU.OQ
Processed company 25/505
PAYX.OQ
Processed company 26/505
AMD.OQ
Processed company 27/505
REG.OQ
Processed company 28/505
DHI.N
Processed company 29/505
TRV.N
Processed company 30/505
BIO.N
Processed company 31/505
MOS.N
Processed company 32/505

In [15]:
prices

,OPEN_PRC,TRDPRC_1,RIC
2024-01-01,<NA>,<NA>,POOL.OQ
2024-01-02,394.26,388.15,POOL.OQ
2024-01-03,382.87,373.78,POOL.OQ
2024-01-04,370.33,372.87,POOL.OQ
2024-01-05,371.6,377.73,POOL.OQ
...,...,...,...
2024-02-17,<NA>,<NA>,AVY.N
2024-02-18,<NA>,<NA>,AVY.N
2024-02-19,<NA>,<NA>,AVY.N
2024-02-20,207.76,209.71,AVY.N


After looking at all the financial data, we can also look at ESG data.

In [16]:
# Initialize an empty DataFrame to store the combined results
all_data_list = []

# Iterate over each chunk and perform the API call
for i, chunk in enumerate(sp500_chunks):
    while True:
        try:
            # Dummy API call function (Replace with actual API call)
            chunk_data = rd.get_data(
            universe=chunk,
            fields=['TR.TRESGScore','TR.EnvironmentPillarScore', 'TR.SocialPillarScore', 'TR.GovernancePillarScore',
                    'TR.TRESGCControversiesScore', 'TR.TRESGScore.date', 'TR.CompanyName'],
            parameters={'SDate': '-3CY', 'EDate': '0CY'}
        )

            # Combine the data
            all_data_list.append(chunk_data)
            break  # Exit the loop if API call is successful

        except Exception as e:
            print(e)
            time.sleep(60)

    # Print the progress
    print(f"Processed chunk {i + 1}/{len(sp500_chunks)}")

/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processed chunk 1/11
Processed chunk 2/11


/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processed chunk 3/11
Processed chunk 4/11
Processed chunk 5/11
Processed chunk 6/11


/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processed chunk 7/11
Processed chunk 8/11


/Users/pascalschrader/Library/CloudStorage/OneDrive-uni-mannheim.de/Mannheim/Lehrstuhl/Lehre/ACC_540/2024/Excercise/Tutorial1/ACC540/venv/lib/python3.11/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processed chunk 9/11
Processed chunk 10/11
Processed chunk 11/11


In [17]:
esg = pd.concat(all_data_list)
esg

,Instrument,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score,ESG Controversies Score,Date,Company Name
0,POOL.OQ,45.07241,10.713089,26.44326,83.441153,100.0,2020-12-31,Pool Corp
1,POOL.OQ,48.927477,12.731237,33.264248,83.866898,100.0,2021-12-31,<NA>
2,POOL.OQ,50.705782,19.995099,35.51464,82.980778,100.0,2022-12-31,<NA>
3,CHRW.OQ,41.077807,31.539665,49.679136,41.34185,100.0,2020-12-31,CH Robinson Worldwide Inc
4,CHRW.OQ,40.228488,25.785817,49.744263,45.189184,100.0,2021-12-31,<NA>
...,...,...,...,...,...,...,...,...
7,J.N,79.321801,76.855787,91.26282,65.790042,100.0,2020-10-02,Jacobs Solutions Inc
8,J.N,80.143392,76.915794,90.845091,68.728316,100.0,2021-10-01,<NA>
9,AVY.N,55.607099,67.9274,64.602871,21.473136,100.0,2019-12-28,Avery Dennison Corp
10,AVY.N,56.217896,66.378613,64.260734,27.009987,100.0,2021-01-02,<NA>
